In [5]:
import torch
import torchvision
from torch import nn
from torch import optim
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import TensorDataset
from torchvision import transforms
import os
import copy

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt



In [6]:
#读取数据
TrainData = 'TrainData.csv'
TestData = 'TestData.csv'

traindata = pd.read_csv(TrainData)
testdata = pd.read_csv(TestData)

trainData = np.array(traindata)[:,:4]
trainLable = np.array(traindata)[:,-1]
trainData = torch.tensor(trainData).float()
trainLable = torch.tensor(trainLable).long()#改成long形式，不然后面会出错

testData = np.array(testdata)[:,:4]
testLable = np.array(testdata)[:,-1]
testData = torch.tensor(testData).float()
testLable = torch.tensor(testLable).long()

epochs = 200#迭代200次
batch = 10#最开始batchsize设置为10
####参考，对tensor进行打包合并
train_ds = TensorDataset(trainData,trainLable)
###参考，打乱训练集数据顺序，希望模型不关注顺序
train = DataLoader(train_ds,batch_size = batch,shuffle = True)

test_ds = TensorDataset(testData,testLable)
test = DataLoader(train_ds,batch_size = batch)

In [7]:
##创建模型
##继承nn.Module这个类并自定义模型
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        ##输入到隐藏层，4个输入特征，64个输出特征
        self.hidden = nn.Sequential(nn.Linear(4,64),nn.ReLU(inplace = True))
        ##输入到输出层，64个输入特征，3个输出特征
        self.predict = nn.Linear(64,3)

    def forward(self,input):##搭建的前馈神经网络,调用上述的层处理输入
        ##隐藏层对输入进行调用，并使用激活函数
        x = self.hidden(input)
        ##多分类不用激活
        out = self.predict(x)
        
        return out
    
net = Net()

In [8]:
##损失函数
loss_func = nn.CrossEntropyLoss()##交叉熵

###准确率、混淆矩阵……
def accuracy(test_out,testLable):
    test_predict = torch.argmax(test_out,dim = 1)#?
    accuracy = (test_predict == testLable).float().mean()
    return accuracy

In [9]:
##优化算法
from torch import optim
optimizer = optim.Adam(net.parameters(),lr = 0.0001)

In [ ]:
##迭代

##设置四个列表储存每次迭代时训练集和测试集的准确率和损失
train_loss=[]
train_accuracy=[]
test_loss=[]
test_accuracy=[]

for epoch in range(epochs):
    for x,y in train:#遍历每一个iteration
        y_out = net(x)
        loss = loss_func(y_out,y)#计算每一个iteration的损失
        optimizer.zero_grad()#梯度设为0
        loss.backward()#反向传播求解梯度
        #优化
        optimizer.step()
    with torch.no_grad():
        trainAccuracy = accuracy(net(trainData),trainLable)
        trainLoss = loss_func(net(trainData),trainLable).data
        testAccuracy = accuracy(net(testData),testLable)
        testLoss = loss_func(net(testData), testLable).data
        print('epoch: ',epoch,'train_loss: ',round(trainLoss.item(),3),'train_accuracy: ',round(trainAccuracy.item(),3),
              'test_loss: ',round(testLoss.item(),3),'test_accuracy: ',round(testAccuracy.item(),3)
              )
        train_loss.append(trainLoss)
        train_accuracy.append(trainAccuracy)
        test_loss.append(testLoss)
        test_accuracy.append(testAccuracy)
        
plt.plot(range(1,epochs+1),train_loss,label='train_loss')
plt.plot(range(1,epochs+1),test_loss,label='test_loss')
plt.plot(range(1,epochs+1),train_acc,label='train_acc')
plt.plot(range(1,epochs+1),test_acc,label='test_acc')
plt.show()


epoch:  0 train_loss:  1.223 train_accuracy:  0.0 test_loss:  1.209 test_accuracy:  0.0
epoch:  1 train_loss:  1.201 train_accuracy:  0.0 test_loss:  1.189 test_accuracy:  0.0
epoch:  2 train_loss:  1.181 train_accuracy:  0.0 test_loss:  1.171 test_accuracy:  0.0
epoch:  3 train_loss:  1.164 train_accuracy:  0.0 test_loss:  1.155 test_accuracy:  0.0
epoch:  4 train_loss:  1.147 train_accuracy:  0.217 test_loss:  1.139 test_accuracy:  0.267
epoch:  5 train_loss:  1.132 train_accuracy:  0.333 test_loss:  1.125 test_accuracy:  0.333
epoch:  6 train_loss:  1.118 train_accuracy:  0.333 test_loss:  1.112 test_accuracy:  0.333
epoch:  7 train_loss:  1.104 train_accuracy:  0.333 test_loss:  1.099 test_accuracy:  0.333
epoch:  8 train_loss:  1.091 train_accuracy:  0.333 test_loss:  1.087 test_accuracy:  0.333
epoch:  9 train_loss:  1.078 train_accuracy:  0.333 test_loss:  1.075 test_accuracy:  0.333
epoch:  10 train_loss:  1.065 train_accuracy:  0.333 test_loss:  1.063 test_accuracy:  0.333
epo

epoch:  91 train_loss:  0.518 train_accuracy:  0.867 test_loss:  0.529 test_accuracy:  0.9
epoch:  92 train_loss:  0.515 train_accuracy:  0.867 test_loss:  0.526 test_accuracy:  0.9
epoch:  93 train_loss:  0.512 train_accuracy:  0.867 test_loss:  0.523 test_accuracy:  0.9
epoch:  94 train_loss:  0.51 train_accuracy:  0.858 test_loss:  0.521 test_accuracy:  0.9
epoch:  95 train_loss:  0.507 train_accuracy:  0.867 test_loss:  0.518 test_accuracy:  0.9
epoch:  96 train_loss:  0.505 train_accuracy:  0.867 test_loss:  0.515 test_accuracy:  0.9
epoch:  97 train_loss:  0.502 train_accuracy:  0.867 test_loss:  0.513 test_accuracy:  0.9
epoch:  98 train_loss:  0.5 train_accuracy:  0.867 test_loss:  0.51 test_accuracy:  0.9
epoch:  99 train_loss:  0.497 train_accuracy:  0.867 test_loss:  0.508 test_accuracy:  0.9
epoch:  100 train_loss:  0.495 train_accuracy:  0.867 test_loss:  0.505 test_accuracy:  0.9
epoch:  101 train_loss:  0.493 train_accuracy:  0.867 test_loss:  0.503 test_accuracy:  0.9
e

epoch:  192 train_loss:  0.346 train_accuracy:  0.967 test_loss:  0.344 test_accuracy:  1.0
epoch:  193 train_loss:  0.344 train_accuracy:  0.958 test_loss:  0.343 test_accuracy:  1.0
epoch:  194 train_loss:  0.343 train_accuracy:  0.958 test_loss:  0.342 test_accuracy:  1.0
epoch:  195 train_loss:  0.342 train_accuracy:  0.958 test_loss:  0.34 test_accuracy:  1.0
epoch:  196 train_loss:  0.341 train_accuracy:  0.958 test_loss:  0.339 test_accuracy:  1.0
epoch:  197 train_loss:  0.34 train_accuracy:  0.967 test_loss:  0.338 test_accuracy:  1.0
epoch:  198 train_loss:  0.339 train_accuracy:  0.95 test_loss:  0.336 test_accuracy:  1.0
epoch:  199 train_loss:  0.337 train_accuracy:  0.95 test_loss:  0.335 test_accuracy:  1.0
